# Ноутбук с разведочным анализом данных по квартирам, сдающимся в аренду

### Установим и импортируем необходимые библиотеки

In [ ]:
!pip install numpy matplotlib seaborn -q

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')

In [2]:
pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 30)

#### Загрузим данные и посмотрим на них

In [46]:
data = pd.read_csv('/home/seppoku/ds_bootcamp/real_estate_rent/data/raw/_data.csv')
data.head()

#### Удалим столбец 'Unnamed: 0'

In [47]:
data.drop(['Unnamed: 0'], axis=1, inplace=True)

#### Возьмем колонки до столбца 'ремонт' (так поделились) и назовем колонки по английски

In [48]:
df_1 = data[['ID  объявления', 'Количество комнат', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Дом', 'Парковка', 'Цена', 'Телефоны', 'Описание']]

In [49]:
df_1.columns = ['announcement_id', 'rooms', 'type', 'subway', 'adress', 'square', 'building', 'parking', 'price', 'contact_phone', 'description']

### Посмотрим признак rooms

In [52]:
df_1['rooms'].value_counts()
# оставим только количество комнат, если там пропуск, заменим на 'свободная'
df_1['rooms'] = df_1['rooms'].fillna('Свободная')
df_1['rooms'] = [i.split(', ')[0] for i in df_1['rooms']]


In [53]:
def plot_bar_chart(df, column_name):
    fig, ax = plt.subplots(figsize=(10, 6))

    sns.countplot(data=df, x=column_name, ax=ax)

    ax.set_ylabel('Количество')
    ax.set_title(f'Распределение значений для {column_name}')


plot_bar_chart(df_1, 'rooms')

##### Видим, что чаще всего сдаются в аренду два пласта - **однушка** и **двушка**

### Посмотрим признак 'parking'

In [54]:
# заменим пропуски на 'во дворе'
df_1['parking'] = df_1['parking'].fillna('во дворе')
plot_bar_chart(df_1, 'parking')

##### Тут видим, что дома обычно не имеют отдельно построенных парковок, они в основном назменые - либо для **жильцов** либо для **всех кто захочет**

### Рассмотрим признак square

In [62]:
#сделаем новый столбец all_square, который будет в себя включать общую площадь квартиры
#df['all_square'] = [i.split('/')[0] for i in df['square']]
#df['all_square'] = df['all_square'].astype(float)
sns.histplot(df_1['all_square'], kde=True)

##### Мы можем видеть, что большинство квартир находятся в диапазоне от 40 до 70 метров квадратных


In [63]:
sns.boxplot(df_1['all_square'])

##### Медианное значение общей площади находится в районе 50, так же есть много квартир 150+ метров, это либо для тусовок посуточно, либо для аренды залов на мероприятия(конференции, свадьбы и тд) надо разбираться

### Посмотрим признак 'type'

In [64]:
plot_bar_chart(df_1, 'type')

##### Здесь всего одно значение - **квартира**

### Посмотрим признак 'building'

In [79]:
# Создадим столбец 'floor', где будет этаж
df_1['floor'] = [int(i.split('/')[0]) for i in df_1['building']]
sns.histplot(df_1['floor'])

##### Большинство квартир находится на высоте от 1 до 15 этажа

In [80]:
# Создадим столбец 'building_floor', где будет этажность самого здания
df_1['buildind_floor'] = [int(i.split('/')[1].split(',')[0]) for i in df_1['building']]
sns.histplot(df_1['buildind_floor'])

##### Большинство домов высотой - 9, 5, 12, 17, 16 скорее всего это так называемые **панельки** и **сталинки**

In [89]:
# Создадим столбец 'building_type', где будет тип постройки, если там пропуск - пусть там будет "Загадка"
df_1['building_type'] = [i.split(', ')[-1] if len(i.split(', ')[-1]) > 6 else 'Загадка' for i in df_1['building']]


fig, ax = plt.subplots(figsize=(10, 6))
sns.histplot(df_1['building_type'], ax=ax);
ax.set_xticklabels(df_1['building_type'].unique(), rotation=90);



##### Больше всего **панелек**, **монолитных** и **кирпичных строений**

## Посмотрим признак цена

In [91]:
# создадим колонку 'price_per_month' в которой будет отображена стоимость месячной аренды
df_1['price_per_month'] = [int(i.split('.')[0]) for i in df_1['price']]
sns.histplot(df_1['price_per_month']);

In [93]:
df_1.describe()

#### Видим, что минимальная цена аренды 5000 - это либо комната, либо цена в валюте, надо разбираться

In [94]:
sns.scatterplot(x = df_1['all_square'], y=df_1['price_per_month'])


In [96]:
fig, ax = plt.subplots(figsize=(10, 6))
sns.scatterplot(x = df_1['building_type'], y=df_1['price_per_month'], ax=ax)
ax.set_xticklabels(df_1['building_type'].unique(), rotation=90);

In [103]:
df_corr = df_1[['all_square', 'floor', 'buildind_floor', 'price_per_month']].corr()

sns.heatmap(df_corr, cmap="crest", annot=True)

##### Видим достаточно сильную корреляцию между ценой и общей площадью

# Небольшой общий вывод:  На первый взгялд данные распределены более менее нормально, с левыми пиками. Выбросы есть, с ними надо будет работать